In [1]:
# import moduel은 그냥 복붙한거라서 어떤게 맞을지 잘 모릅니다

import numpy as np
import pandas as pd
from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from tqdm import tnrange, tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import catboost
from catboost import CatBoostClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
def fill_na_mode(data):                   # fill with mode. (최빈값 fillna)
    fill_null_list = list(data.columns)
    fill_null_list.remove('TIPI10')
    for col in fill_null_list:
        data[col].fillna(data[col].mode()[0],inplace=True)
    data['TIPI10'].fillna(1,inplace=True)

    return data

def encoding_label(data):                          # Encoding Label
    encoder = LabelEncoder()
    label_encoded = encoder.fit_transform(data['country'])
    data['country'] = label_encoded    
  
    return data

def del_outlier(data):
    q_list = [i for i in data.columns if "Q" in i]
    q_drop_index = train_df[data[q_list].isnull().sum(1)==26].index
    tipi_list = [i for i in data.columns if "TIPI" in i]
    tipi_drop_index = data[data[tipi_list].isnull().sum(1)>3].index
    familysize_drop_index = data[data['familysize']>10].index
    age_drop_index = data[data['age']>=100].index

    drop_index = list(tipi_drop_index)+list(q_drop_index)+list(familysize_drop_index)+list(age_drop_index)
    data.drop(drop_index,inplace = True)
    data.reset_index(drop = True,inplace=True)

    return data

def processing_data(data, idx = "train"):      # 데이터 전처리 함수
    data = fill_na_mode(data)
    data = encoding_label(data)

    if idx == 'train':
        data = del_outlier(data)
    return data

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
# 경로는 개인에 맞게 맞춰서 조정하시면 됩니다

train_df = pd.read_csv('./competition_data/train.csv')
test_df = pd.read_csv('./competition_data/test.csv')
sample_submission = pd.read_csv('./competition_data/sample_submission.csv')

train_df = processing_data(train_df)
test_df = processing_data(test_df, idx = "test")

train_Y = train_df['nerdiness']
train_X = train_df.drop('nerdiness', axis = 1)

In [39]:
params = {'lambda': 9.052528986664745, 
          'alpha': 0.0010381252728911819, 
          'colsample_bytree': 0.3, 
          'subsample': 1.0, 
          'learning_rate': 0.016, 
          'max_depth': 5, 
          'random_state': 2020, 
          'min_child_weight': 11}
xgb_model = xgb.XGBClassifier(**params)
xgb_model.fit(train_X, train_Y)

y_pred = xgb_model.predict(test_df)
sample_submission['nerdiness'] = y_pred

In [40]:
y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [41]:
sample_submission.to_csv('./sample_submission.csv', index=None)

In [ ]:
sam